In [2]:
base_url='https://e4ftl01.cr.usgs.gov/MOLT/MOD16A2GF.061/'
date='2024.12.26'


In [3]:
import os
import requests
from bs4 import BeautifulSoup
from requests.auth import HTTPBasicAuth

# Set Earthdata login credentials
USERNAME = "trngbich"
PASSWORD = "N3t8n1994"

# MOD16A2GF Data Pool URL
url = "https://e4ftl01.cr.usgs.gov/MOLT/MOD16A2GF.061/2024.12.26/"

# Create output directory
output_dir = "E:/MOD16A2GF_hdf"
os.makedirs(output_dir, exist_ok=True)

# Get list of files
session = requests.Session()
response = session.get(url, auth=HTTPBasicAuth(USERNAME, PASSWORD))

# Parse HTML
soup = BeautifulSoup(response.text, "html.parser")
files = [node.get("href") for node in soup.find_all("a") if node.get("href").endswith(".hdf")]

# Download each file
for file in files:
    file_url = url + file
    local_path = os.path.join(output_dir, file)
    
    print(f"Downloading {file_url} ...")
    file_response = session.get(file_url, auth=HTTPBasicAuth(USERNAME, PASSWORD), stream=True)
    
    with open(local_path, "wb") as f:
        for chunk in file_response.iter_content(chunk_size=1024):
            f.write(chunk)
    
    print(f"Saved: {local_path}")

print("All HDF files downloaded successfully!")


Saved: E:/MOD16A2GF_hdf\MOD16A2GF.A2024361.h00v08.061.2025019223305.hdf
Saved: E:/MOD16A2GF_hdf\MOD16A2GF.A2024361.h00v09.061.2025019224502.hdf
Saved: E:/MOD16A2GF_hdf\MOD16A2GF.A2024361.h00v10.061.2025019223405.hdf
Saved: E:/MOD16A2GF_hdf\MOD16A2GF.A2024361.h01v08.061.2025019223252.hdf
Saved: E:/MOD16A2GF_hdf\MOD16A2GF.A2024361.h01v09.061.2025019223545.hdf
Saved: E:/MOD16A2GF_hdf\MOD16A2GF.A2024361.h01v10.061.2025019222926.hdf
Saved: E:/MOD16A2GF_hdf\MOD16A2GF.A2024361.h01v11.061.2025019224746.hdf
Saved: E:/MOD16A2GF_hdf\MOD16A2GF.A2024361.h02v06.061.2025019223740.hdf
Saved: E:/MOD16A2GF_hdf\MOD16A2GF.A2024361.h02v08.061.2025019224413.hdf
Saved: E:/MOD16A2GF_hdf\MOD16A2GF.A2024361.h02v09.061.2025019222959.hdf
Saved: E:/MOD16A2GF_hdf\MOD16A2GF.A2024361.h02v10.061.2025019225000.hdf
Saved: E:/MOD16A2GF_hdf\MOD16A2GF.A2024361.h02v11.061.2025019223646.hdf
Saved: E:/MOD16A2GF_hdf\MOD16A2GF.A2024361.h03v06.061.2025019224028.hdf
Saved: E:/MOD16A2GF_hdf\MOD16A2GF.A2024361.h03v07.061.2025019224

In [1]:
import os
import glob
import h5py
import rasterio
from osgeo import gdal

# Directory where your HDF files are stored
hdf_dir = "MOD16A2GF_data"
output_dir = "MOD16A2GF_geotiff"

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Find all HDF files
hdf_files = glob.glob(os.path.join(hdf_dir, "*.hdf"))

def convert_hdf_to_geotiff(hdf_file, output_dir):
    """Convert MOD16A2GF HDF file to GeoTIFF"""
    hdf_ds = gdal.Open(hdf_file, gdal.GA_ReadOnly)
    subdatasets = hdf_ds.GetSubDatasets()[0][0]
    
    for i, subdataset in enumerate(subdatasets):
        subdataset_name = subdataset[0]  # Get dataset path
        layer_name = subdataset_name.split(":")[-1]  # Extract layer name

        output_tif = os.path.join(output_dir, f"{os.path.basename(hdf_file).replace('.hdf', '')}_{layer_name}.tif")
        
        # Convert using GDAL Translate
        gdal.Translate(output_tif, subdataset_name, format="GTiff")

        print(f"Converted: {output_tif}")

# Process all HDF files
for hdf_file in hdf_files:
    convert_hdf_to_geotiff(hdf_file, output_dir)

print("All HDF files converted to GeoTIFFs.")


C:\Users\ntr002\.conda\envs\phd\Lib\site-packages\osgeo\gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


AttributeError: 'NoneType' object has no attribute 'GetSubDatasets'

In [ ]:
from rasterio.merge import merge
import rasterio

# List all GeoTIFF files
tif_files = glob.glob(os.path.join(output_dir, "*.tif"))

# Read all GeoTIFFs
src_files = [rasterio.open(f) for f in tif_files]

# Merge
mosaic, out_transform = merge(src_files)

# Save as global GeoTIFF
out_meta = src_files[0].meta.copy()
out_meta.update({"height": mosaic.shape[1], "width": mosaic.shape[2], "transform": out_transform})

with rasterio.open(os.path.join(output_dir, "global_MOD16A2GF.tif"), "w", **out_meta) as dest:
    dest.write(mosaic)

print("Global MOD16A2GF GeoTIFF created.")


In [6]:
import requests
from requests.auth import HTTPBasicAuth

# URL of the file to download
url = 'https://e4ftl01.cr.usgs.gov/MOLT/MOD16A2GF.061/2024.12.26/MOD16A2GF.A2024361.h00v08.061.2025019223305.hdf'

# Authentication credentials
username = "trngbich"
password = "N3t8n1994"

# Destination file path
destination_path = r'E:/MOD16A2GF_hd/MOD16A2GF.A2024361.h00v08.061.2025019223305.hdf'

# Download the file
response = requests.get(url, auth=HTTPBasicAuth(username, password), stream=True)
response.raise_for_status()  # Check if the request was successful

# Save the file locally
with open(destination_path, 'wb') as file:
    for chunk in response.iter_content(chunk_size=8192):
        file.write(chunk)

print(f'File downloaded and saved to {destination_path}')


HTTPError: 401 Client Error: Unauthorized for url: https://urs.earthdata.nasa.gov/oauth/authorize?scope=uid&app_type=401&client_id=ijpRZvb9qeKCK5ctsn75Tg&response_type=code&redirect_uri=https%3A%2F%2Fe4ftl01.cr.usgs.gov%2Foauth&state=aHR0cHM6Ly9lNGZ0bDAxLmNyLnVzZ3MuZ292L01PTFQvTU9EMTZBMkdGLjA2MS8yMDI0LjEyLjI2L01PRDE2QTJHRi5BMjAyNDM2MS5oMDB2MDguMDYxLjIwMjUwMTkyMjMzMDUuaGRm

In [7]:
import requests
from getpass import getpass
from requests.auth import HTTPBasicAuth

# Set Earthdata login credentials
username = "trngbich"
password =  "N3t8n1994"

# URL of the file to download
url = 'https://e4ftl01.cr.usgs.gov/MOLT/MOD16A2GF.061/2024.12.26/MOD16A2GF.A2024361.h23v02.061.2025019224914.hdf'

# Destination file path
destination_path = r'E:/MOD16A2GF_hd/MOD16A2GF.A2024361.h00v08.061.2025019223305.hdf'

# Authenticate and create a session
session = requests.Session()
session.auth = (username, password)

# Perform the authenticated request to the file
response = session.get(url, stream=True)
response.raise_for_status()  # Check if the request was successful

# Save the file locally
with open(destination_path, 'wb') as file:
    for chunk in response.iter_content(chunk_size=8192):
        file.write(chunk)

print(f'File downloaded and saved to {destination_path}')


HTTPError: 401 Client Error: Unauthorized for url: https://urs.earthdata.nasa.gov/oauth/authorize?scope=uid&app_type=401&client_id=ijpRZvb9qeKCK5ctsn75Tg&response_type=code&redirect_uri=https%3A%2F%2Fe4ftl01.cr.usgs.gov%2Foauth&state=aHR0cHM6Ly9lNGZ0bDAxLmNyLnVzZ3MuZ292L01PTFQvTU9EMTZBMkdGLjA2MS8yMDI0LjEyLjI2L01PRDE2QTJHRi5BMjAyNDM2MS5oMDB2MDguMDYxLjIwMjUwMTkyMjMzMDUuaGRm

In [8]:
session = requests.Session()
session.auth = (username, password)

In [9]:
session

In [10]:
response = session.get(url, stream=True)

In [11]:
response

<Response [401]>

In [12]:
from ftplib import FTP

# FTP server details
ftp_host = 'https://e4ftl01.cr.usgs.gov/'
ftp_user = 'trngbich'
ftp_pass = 'N3t8n1994'

# Connect to the FTP server
ftp = FTP(ftp_host)
ftp.login(user=ftp_user, passwd=ftp_pass)
print(f'Connected to FTP server: {ftp_host}')



gaierror: [Errno 11001] getaddrinfo failed

In [13]:
from pymodis import downmodis

In [16]:
modisOgg = downmodis.downModis(url="https://e4ftl01.cr.usgs.gov", 
                               user='trngbich',
                               password='N3t8n1994',
                               # token=token,
                               destinationFolder='E:/MOD16A2GF_hdf',
                               # tiles=options.tiles, path=options.path,
                               product='MOD16A2GF.061', 
                               today='2018-01-01',
                               enddate='2024-12-31', 
                               # jpg=options.jpg,
                               # delta=int(options.delta),
                               # debug=options.debug
                              )
# connect to ftp
modisOgg.connect()
if modisOgg.nconnection <= 20:
    # download data
    modisOgg.downloadsAllDay()
else:
    parser.error("A problem with the connection occured")


AttributeError: 'KeyError' object has no attribute 'code'